# TODO

* use https://colab.research.google.com/drive/1iDHCYIrWswIKp-n-pOg69xLoZO09MEgf#scrollTo=zjzTkJGl1J0l this model and arch instead

In [1]:
!pip install pandas
!pip install tqdm
!pip install torch
!pip install pytorch_pretrained_bert
!pip install pytorch-ignite

In [1]:
import requests
import logging
import tarfile
import urllib
from tqdm import tqdm as tqdm

logger = logging.getLogger()

import sys
import os
from pathlib import Path

In [2]:
DATA_DIR = Path('./data').resolve()
IMDB = DATA_DIR/'aclImdb'

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

def download_url(url:str, dest:str, overwrite:bool=True, show_progress=True, 
                 chunk_size=1024*1024, timeout=4, retries=5)->None:
    "Download `url` to `dest` unless it exists and not `overwrite`."
    dest = Path(dest)/os.path.basename(url)
    if os.path.exists(dest) and not overwrite: 
        print("File already existing")
        return

    s = requests.Session()
    s.mount('http://',requests.adapters.HTTPAdapter(max_retries=retries))
    u = s.get(url, stream=True, timeout=timeout)
    try: file_size = int(u.headers["Content-Length"])
    except: show_progress = False
    print(f"Downloading {url}")
    with open(dest, 'wb') as f:
        nbytes = 0
        if show_progress: 
            pbar = tqdm(range(file_size), leave=False)
        try:
            for chunk in u.iter_content(chunk_size=chunk_size):
                nbytes += len(chunk)
                if show_progress: pbar.update(nbytes)
                f.write(chunk)
        except requests.exceptions.ConnectionError as e:
            print(f"Download failed after {retries} retries.")
            import sys;sys.exit(1)
        finally:
            return str(dest)
        
def untar(file_path, dest:str):
    print(f"Untar {os.path.basename(file_path)} to {dest}")
    with tarfile.open(file_path) as tf:
        tf.extractall(path=str(dest))
    os.remove(file_path)
    return str(dest)

## Download imdb data

In [12]:
file_path = download_url(url, '/tmp', overwrite=True)
untar(file_path, DATA_DIR)

Untar aclImdb_v1.tar.gz to /workspace/data


'/workspace/data'

In [4]:
!ls -lh $DATA_DIR

total 63M
drwxr-xr-x. 4 root root 105 Jun 26  2011 aclImdb
-rw-r--r--. 1 root root 32M Jul  4 14:43 dev.tsv
-rw-r--r--. 1 root root 32M Jul  4 14:43 train.tsv


## Read imdb data

In [3]:
import pandas as pd
import re

def clean_html(raw):
    cleanr = re.compile('<.*?>')
    clean = re.sub(cleanr, '  ', raw)
    return re.sub(' +', ' ', clean)


def read_imdb(imdb_dir: str, text_col='text', label_col='label'):
    "Read imdb data to {'label', 'text'} format"
    imdb_dir = Path(imdb_dir)
    datasets = {}
    
    for t in ['train', 'test']:
        texts, labels = [], []
        for p in ['pos', 'neg']:
            for file in tqdm((imdb_dir/'train'/p).glob("*.txt"), desc=f'reading {t}/{p}'):
                with open(file, 'r') as fin:
                    text = fin.readlines()[0].replace(r'\n', ' ')
                    text = clean_html(text).strip()
                    texts +=  [text]
                    labels += [p]
        df = pd.DataFrame(
            {label_col: labels, text_col: texts})
        datasets[t] = df.sample(frac=1)

    return datasets

In [4]:
datasets = read_imdb(IMDB)

labels = list(set(datasets['train']['label'].tolist()))
label2int = {label: i for i, label in enumerate(labels)}

reading train/pos: 12500it [00:01, 9103.39it/s]
reading train/neg: 12500it [00:01, 9231.60it/s]
reading test/pos: 12500it [00:01, 9245.09it/s]
reading test/neg: 12500it [00:01, 9255.92it/s]


## Configs - args: base model parameters, adapt_args: fine-tuning parameters

In [75]:
from collections import namedtuple
import torch

LOG_DIR = "./logs/"
CACHE_DIR = "./cache/"

NUM_MAX_POSITIONS = 256
BATCH_SIZE = 32

device = "cuda" if torch.cuda.is_available() else "cpu"

AdaptationConfig = namedtuple('AdaptationConfig',
  field_names="num_classes, dropout, initializer_range, batch_size, lr, max_norm, n_epochs,"
              "n_warmup, valid_pct, gradient_accumulation_steps, device,"
              "log_dir, dataset_cache")
adapt_args = AdaptationConfig(
               2          , 0.1    , 0.02             , 32        , 6.5e-5, 1.0   , 3,
               10      , 0.1           , 1, device,
               LOG_DIR, CACHE_DIR+'dataset_cache.bin')

## DataProcessor

In [76]:
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader
import numpy as np
import warnings
from tqdm import tqdm_notebook as tqdm


class DataProcessor:
    
    CLS = '[CLS]'
    PAD = '[PAD]'
    
    def __init__(self, tokenizer, label2id, num_max_positions=512):
        self.tokenizer=tokenizer
        self.label2id = label2id
        self.num_labels = len(label2id)
        self.num_max_positions = num_max_positions
        
    
    def process_example(self, example):
        assert len(example) == 2
        label, text = example[0], example[1]
        assert isinstance(text, str)
        
#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore")
        tokens = self.tokenizer.tokenize(text)
        if len(tokens) >= self.num_max_positions:
            tokens = tokens[:self.num_max_positions-1] 
            ids =  self.tokenizer.convert_tokens_to_ids(tokens) + [self.tokenizer.vocab[self.CLS]]
        else:
            pad = [self.tokenizer.vocab[self.PAD]] * (self.num_max_positions-len(tokens)-1)
            ids =  self.tokenizer.convert_tokens_to_ids(tokens) + [self.tokenizer.vocab[self.CLS]] + pad
        
#         ids =  self.tokenizer.convert_tokens_to_ids(tokens)         
#         if len(ids) >= self.num_max_positions:
#             ids = ids[:self.num_max_positions-1] + [self.tokenizer.vocab[self.CLS]]
#         else:
#             pad = [self.tokenizer.vocab[self.PAD]] * (self.num_max_positions-len(ids)-1)
#             ids = ids + [self.tokenizer.vocab[self.CLS]] + pad
        return ids, self.label2id[label]
    

def create_dataset(df, processor, batch_size=32, shuffle=False, valid_pct=None, 
                   text_col="text", label_col="label"):
    "Process rows in `df` with `processor` and return a DataLoader"
    
    features, labels = [], []
    
    for i, row in tqdm(df.iterrows(), total=len(df)):
        ids, lbl = processor.process_example([row.label, row.text])
        features += [ids]
        labels += [lbl]
    
    dataset = TensorDataset(
                    torch.tensor(features, dtype=torch.long), 
                    torch.tensor(labels, dtype=torch.long))
    
    if valid_pct is not None:
        
        valid_size = int(valid_pct * len(df))
        train_size = len(df) - valid_size
        valid_dataset, train_dataset = random_split(dataset, [valid_size, train_size])
        
        valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)    
        return train_loader, valid_loader

    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return data_loader
    

In [77]:
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
processor = DataProcessor(tokenizer, label2int, num_max_positions=NUM_MAX_POSITIONS)

In [81]:
df_train = datasets["train"].sample(5000)
df_test = datasets["test"].sample(5000)

In [82]:
train_dl, valid_dl = create_dataset(df_train, processor, batch_size=adapt_args.batch_size, 
                                    valid_pct=adapt_args.valid_pct)

test_dl = create_dataset(df_test, processor, batch_size=adapt_args.batch_size, valid_pct=None)

## TransformerWithClfHead

In [83]:
import torch
import torch.nn as nn

def get_num_params(model):
    import numpy as np
    mp = filter(lambda p: p.requires_grad, model.parameters())
    return sum(np.prod(p.size()) for p in mp)

class Transformer(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_embeddings, num_max_positions, num_heads, num_layers, dropout, causal):
        super().__init__()
        self.causal = causal
        self.tokens_embeddings = nn.Embedding(num_embeddings, embed_dim)
        self.position_embeddings = nn.Embedding(num_max_positions, embed_dim)
        self.dropout = nn.Dropout(dropout)

        self.attentions, self.feed_forwards = nn.ModuleList(), nn.ModuleList()
        self.layer_norms_1, self.layer_norms_2 = nn.ModuleList(), nn.ModuleList()
        for _ in range(num_layers):
            self.attentions.append(nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout))
            self.feed_forwards.append(nn.Sequential(nn.Linear(embed_dim, hidden_dim),
                                                    nn.ReLU(),
                                                    nn.Linear(hidden_dim, embed_dim)))
            self.layer_norms_1.append(nn.LayerNorm(embed_dim, eps=1e-12))
            self.layer_norms_2.append(nn.LayerNorm(embed_dim, eps=1e-12))

    def forward(self, x, padding_mask=None):
        """ x has shape [seq length, batch], padding_mask has shape [batch, seq length] """
        positions = torch.arange(len(x), device=x.device).unsqueeze(-1)
        h = self.tokens_embeddings(x)
        h = h + self.position_embeddings(positions).expand_as(h)
        h = self.dropout(h)

        attn_mask = None
        if self.causal:
            attn_mask = torch.full((len(x), len(x)), -float('Inf'), device=h.device, dtype=h.dtype)
            attn_mask = torch.triu(attn_mask, diagonal=1)

        for layer_norm_1, attention, layer_norm_2, feed_forward in zip(self.layer_norms_1, self.attentions,
                                                                       self.layer_norms_2, self.feed_forwards):
            h = layer_norm_1(h)
            x, _ = attention(h, h, h, attn_mask=attn_mask, need_weights=False, key_padding_mask=padding_mask)
            x = self.dropout(x)
            h = x + h

            h = layer_norm_2(h)
            x = feed_forward(h)
            x = self.dropout(x)
            h = x + h
        return h


class TransformerWithClfHead(nn.Module):
    def __init__(self, config, fine_tuning_config):
        super().__init__()
        self.config = fine_tuning_config
        self.transformer = Transformer(config.embed_dim, config.hidden_dim, config.num_embeddings,
                                       config.num_max_positions, config.num_heads, config.num_layers,
                                       fine_tuning_config.dropout, causal=not config.mlm)
        
        self.classification_head = nn.Linear(config.embed_dim, fine_tuning_config.num_classes)

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding, nn.LayerNorm)):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
        if isinstance(module, (nn.Linear, nn.LayerNorm)) and module.bias is not None:
            module.bias.data.zero_()

    def forward(self, x, clf_tokens_mask, clf_labels=None, padding_mask=None):
        hidden_states = self.transformer(x, padding_mask)

        clf_tokens_states = (hidden_states * clf_tokens_mask.unsqueeze(-1).float()).sum(dim=0)
        clf_logits = self.classification_head(clf_tokens_states)

        if clf_labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)
            loss = loss_fct(clf_logits.view(-1, clf_logits.size(-1)), clf_labels.view(-1))
            return clf_logits, loss
        return clf_logits

In [84]:
from pytorch_pretrained_bert import cached_path

state_dict = torch.load(cached_path("https://s3.amazonaws.com/models.huggingface.co/"
                                    "naacl-2019-tutorial/model_checkpoint.pth"), map_location='cpu')

args = torch.load(cached_path("https://s3.amazonaws.com/models.huggingface.co/"
                                        "naacl-2019-tutorial/model_training_args.bin"))

adaptation_model = TransformerWithClfHead(config=args, fine_tuning_config=adapt_args).to(adapt_args.device)

incompatible_keys = adaptation_model.load_state_dict(state_dict, strict=False)
print(f"Parameters discarded from the pretrained model: {incompatible_keys.unexpected_keys}")
print(f"Parameters added in the adaptation model: {incompatible_keys.missing_keys}")

Parameters discarded from the pretrained model: ['lm_head.weight']
Parameters added in the adaptation model: ['classification_head.weight', 'classification_head.bias']


In [85]:
get_num_params(adaptation_model)

50397182

In [86]:
adaptation_model.classification_head

Linear(in_features=410, out_features=2, bias=True)

## Prepare fine-tuning loop

In [87]:
from ignite.engine import Engine, Events
from ignite.metrics import RunningAverage, Accuracy 
from ignite.handlers import ModelCheckpoint
from ignite.contrib.handlers import CosineAnnealingScheduler, PiecewiseLinear, create_lr_scheduler_with_warmup, ProgressBar

optimizer = torch.optim.Adam(adaptation_model.parameters(), lr=adapt_args.lr)

def update(engine, batch):
    
    adaptation_model.train()
    inputs, labels = (t.to(adapt_args.device) for t in batch)
    
    inputs = inputs.transpose(0, 1).contiguous() # [S, B]
    _, loss = adaptation_model(inputs, 
                               clf_tokens_mask = (inputs == tokenizer.vocab[processor.CLS]), 
                               clf_labels=labels)
    loss = loss / adapt_args.gradient_accumulation_steps
    loss.backward()
    
    torch.nn.utils.clip_grad_norm_(adaptation_model.parameters(), adapt_args.max_norm)
    if engine.state.iteration % adapt_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
    return loss.item()

def inference(engine, batch):
    adaptation_model.eval()
    with torch.no_grad():
        batch, labels = (t.to(adapt_args.device) for t in batch)
        inputs = batch.transpose(0, 1).contiguous()
        logits = adaptation_model(inputs,
                                  clf_tokens_mask = (inputs == tokenizer.vocab[processor.CLS]),
                                  padding_mask = (batch == tokenizer.vocab[processor.PAD]))
    return logits, labels
                              
trainer = Engine(update)
evaluator = Engine(inference)

# Attache metric to evaluator & evaluation to trainer: evaluate on valid set after each epoch
Accuracy().attach(evaluator, "accuracy")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(valid_dl)
    print(f"validation epoch: {engine.state.epoch} acc: {100*evaluator.state.metrics['accuracy']}")
          
# Learning rate schedule: linearly warm-up to lr and then to zero
scheduler = PiecewiseLinear(optimizer, 'lr', [(0, 0.0), (adapt_args.n_warmup, adapt_args.lr),
                                              (len(train_dl)*adapt_args.n_epochs, 0.0)])
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)


# Add progressbar with loss
RunningAverage(output_transform=lambda x: x).attach(trainer, "loss")
ProgressBar(persist=True).attach(trainer, metric_names=['loss'])

# Save checkpoints and finetuning config
checkpoint_handler = ModelCheckpoint(adapt_args.log_dir, 'finetuning_checkpoint', 
                                     save_interval=1, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {'mymodel': adaptation_model})
torch.save(args, os.path.join(adapt_args.log_dir, 'fine_tuning_args.bin'))          

## Lets fine-tune on imdb!

In [88]:
%%time 

trainer.run(train_dl, max_epochs=1)

validation epoch: 1 acc: 84.39999999999999
CPU times: user 23.2 s, sys: 41.7 s, total: 1min 4s
Wall time: 1min 16s


In [89]:
evaluator.run(test_dl)
print(f"test results - acc: {100*evaluator.state.metrics['accuracy']:.3f}")

test results - acc: 86.860


In [151]:
import torch.nn.functional as F


def predict(model, tokenizer, int2label, input="test"):
    tok = tokenizer.tokenize(input)
    ids = tokenizer.convert_tokens_to_ids(tok) + [tokenizer.vocab['[CLS]']]
    tensor = torch.tensor(ids, dtype=torch.long)
    tensor = tensor.to(device)
    tensor = tensor.reshape(1, -1)
    tensor_in = tensor.transpose(0, 1).contiguous() # [S, 1]
    logits = adaptation_model(tensor_in,
                              clf_tokens_mask = (tensor_in == tokenizer.vocab['[CLS]']),
                              padding_mask = (tensor == tokenizer.vocab['[PAD]']))
    val, _ = torch.max(logits, 0)
    val = F.softmax(val, dim=0).detach().cpu().numpy()    
    return {int2label[val.argmax()]: val.max(),
            int2label[val.argmin()]: val.min()}

In [152]:
int2label = {i:label for label,i in label2int.items()}

predict(adaptation_model, tokenizer, int2label, input = "This movie sucks...SO SHIT")

{'neg': 0.5452621, 'pos': 0.45473787}

In [144]:
predict(adaptation_model, tokenizer, int2label, input = "This movie sucks...SO SHIT")

0

In [135]:
val = F.softmax(val, dim=0).detach().cpu().numpy()


In [138]:
ind = ind.detach().cpu().numpy()

for v,i in zip(val, ind):
    print(v,i)

0.5452621 0
0.45473787 0


In [117]:
val

array([0.19339132, 0.01184591], dtype=float32)